# Title

## Introduction

**We would like to know which "kinds" of players are most likely to contribute a large amount of data so that we can target those players in our recruiting efforts.**

## Methods and Results

## Discussion

## References

# Plan:

## Step 1: Model Which age of each gender plays the most
## Step 1.5: Write, write, write.
## Step 2: Bar plot, y axis played hours, x axis a mix of variables attained from the model

# Computation: 

In [199]:
import altair as alt
import numpy as np
import pandas as pd
from sklearn import set_config
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

In [200]:
url = 'https://drive.google.com/file/d/1Mw9vW0hjTJwRWx0bDXiSpYsO3gKogaPz/edit'
path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
players = pd.read_csv(path)
players

,experience,subscribe,hashedEmail,played_hours,name,gender,age,individualId,organizationName
0,Pro,True,f6daba428a5e19a3d47574858c13550499be23603422e6...,30.3,Morgan,Male,9,NaN,NaN
1,Veteran,True,f3c813577c458ba0dfef80996f8f32c93b6e8af1fa9397...,3.8,Christian,Male,17,NaN,NaN
2,Veteran,False,b674dd7ee0d24096d1c019615ce4d12b20fcbff12d79d3...,0.0,Blake,Male,17,NaN,NaN
3,Amateur,True,23fe711e0e3b77f1da7aa221ab1192afe21648d47d2b4f...,0.7,Flora,Female,21,NaN,NaN
4,Regular,True,7dc01f10bf20671ecfccdac23812b1b415acd42c2147cb...,0.1,Kylie,Male,21,NaN,NaN
...,...,...,...,...,...,...,...,...,...
191,Amateur,True,b6e9e593b9ec51c5e335457341c324c34a2239531e1890...,0.0,Bailey,Female,17,NaN,NaN
192,Veteran,False,71453e425f07d10da4fa2b349c83e73ccdf0fb3312f778...,0.3,Pascal,Male,22,NaN,NaN
193,Amateur,False,d572f391d452b76ea2d7e5e53a3d38bfd7499c7399db29...,0.0,Dylan,Prefer not to say,17,NaN,NaN
194,Amateur,False,f19e136ddde68f365afc860c725ccff54307dedd13968e...,2.3,Harlow,Male,17,NaN,NaN


something something, first we must pull the required dataset from the web.

In [201]:
players_clean= players.drop(columns= {"hashedEmail", "individualId", "organizationName"})
players_clean.groupby('experience').mean('age')

,subscribe,played_hours,age
experience,,,
Amateur,0.714286,6.017460,20.253968
Beginner,0.771429,1.248571,21.657143
Pro,0.714286,2.600000,22.214286
Regular,0.805556,18.208333,22.777778
Veteran,0.687500,0.647917,20.958333


In [202]:
p_numb_gender = players_clean.replace(['Male', 'Female', 'Agender', 'Non-binary', 'Two-Spirited', 'Prefer not to say', 'Other'], [1000, 2000, 3000, 4000, 5000, 6000, 7000])
#p_numb_experience
clean = p_numb_gender.sort_values('age', ascending=False).tail(194)
p_numb_clean = clean.sort_values('played_hours', ascending=False).tail(170)


p_numb_plot = alt.Chart(p_numb_clean).mark_point(opacity=0.5).encode(
    x=alt.X('age').title('Age'),
    y=alt.Y('gender').title('Hours Played'),
)
p_numb_plot


/tmp/ipykernel_102/1167789380.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  p_numb_gender = players_clean.replace(['Male', 'Female', 'Agender', 'Non-binary', 'Two-Spirited', 'Prefer not to say', 'Other'], [1000, 2000, 3000, 4000, 5000, 6000, 7000])


alt.Chart(...)

In [203]:
set_config(transform_output='pandas')

preprocessor = make_column_transformer(
    (StandardScaler(), ('age', 'gender')),
    verbose_feature_names_out=False)

ks = range(1, 10)
wssds = [
    make_pipeline(
    	preprocessor,
    	KMeans(n_clusters=k)  # Create a new KMeans model with `k` clusters
    ).fit(p_numb_clean)[1].inertia_
    for k in ks
]

p_clust_ks = pd.DataFrame({
    "k": ks,
    "wssd": wssds,
})

ks_plot = alt.Chart(p_clust_ks).mark_line(point=True).encode(
    x=alt.X('k'),
    y=alt.Y('wssd')
)
ks_plot

kmeans = KMeans(6, random_state=1110)
p_clust = make_pipeline(preprocessor, kmeans)
p_clust.fit(p_numb_clean)
p_cluster = p_numb_clean.assign(cluster = p_clust[1].labels_)
p_clust_plot = alt.Chart(p_cluster).mark_point().encode(
    x=alt.X('gender'),
    y=alt.Y('age'),
    color=alt.Color('cluster:N')
)
p_clust_plot

alt.Chart(...)

In [179]:
#Plot Based on Experience
player_data_experience_mean = (players_clean.groupby('experience')
                        .mean(numeric_only = True)
                        .reset_index()
                              )

player_data_experience_bar = alt.Chart(player_data_experience_mean).mark_bar().encode(
    x=alt.X('experience', title = 'Experience Level'),
    y=alt.Y('played_hours', title = "Average Hours Played (Hours)")
).properties(
    title= 'Average Number of Hours Played by Different Experience Levels'
)

#Plot Based on Age
player_data_age_mean = (players_clean[(players_clean["age"] >= 15) & (players_clean["age"] <= 30)]
                        .groupby('age')
                        .mean(numeric_only = True)
                        .reset_index()
                       )


player_data_age_bar = alt.Chart(player_data_age_mean).mark_bar().encode(
    x=alt.X('age', title = 'Age of Players (Years)'),
    y=alt.Y('played_hours', title = "Average Hours Played (Hours)")
).properties(
    title= 'Average Number of Hours Played by Players of Different Ages'
)

#Plot Based on Gender
player_data_gender_mean = (players_clean.groupby('gender')
                        .mean(numeric_only = True)
                        .reset_index()
                       )

player_data_gender_bar = alt.Chart(player_data_gender_mean).mark_bar().encode(
    x=alt.X('gender', title = 'Reported Gender of Players'),
    y=alt.Y('played_hours', title = 'Average Hours Played (Hours)')
).properties(
    title = 'Average Number of Hours Played by Players of Different Genders'
)

#Plot Based on Subscription Status
player_data_subscribe_mean = (players_clean.groupby('subscribe')
                        .mean(numeric_only = True)
                        .reset_index()
                          )

player_data_subscribe_bar = alt.Chart(player_data_subscribe_mean).mark_bar().encode(
    x=alt.X('subscribe', title = 'Subscription Status of Players'),
    y=alt.Y('played_hours', title = 'Average Hours Played (Hours)')
).properties(
    title = 'Average Number of Hours Played by Players of Different Subscription Status'
)

In [8]:
display(player_data_experience_bar)
display(player_data_age_bar)
display(player_data_gender_bar)
display(player_data_subscribe_bar)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

alt.Chart(...)